In [ ]:
cd ../..

# Totaljobs/CWjobs Spider

In [ ]:
import os
output_file = 'data/raw/totaljobs.csv'
if os.path.exists(output_file):
    os.remove(output_file)

In [ ]:
# %%writefile src/ocean_jobs/data/spiders/totaljobs.py
import scrapy
from bs4 import BeautifulSoup
import html

In [ ]:
# %%writefile -a src/ocean_jobs/data/spiders/totaljobs.py

class JobSpider(scrapy.Spider):
    name = "jobs"
    allowed_domains = ['www.cwjobs.co.uk', 'www.totaljobs.com']
    start_urls = []
    NUM_PAGES = 5

    def start_requests(self):
        urls = [
            'https://www.cwjobs.co.uk/jobs/data-scientist/in-london?radius=10&s=header',
            'https://www.totaljobs.com/jobs/data-scientist/in-london?radius=10'
        ]
        for url in urls:
            yield scrapy.Request(url=url, callback=self.parse1)
            

    def parse1(self, response):
        page_num = response.css(".results-footer-links-container nav span.active::text").get()
        print(page_num)
        next_page = response.url.split("?")[0]+response.css(".results-footer-links-container nav a.btn.btn-default.next::attr(href)").extract()[0]
        
        if int(page_num) <= 5:
            
            
            for job in response.css("div.job-title"):
                yield scrapy.Request(url=str(job.css("a::attr(href)").get()), callback=self.parse2)
                
            yield scrapy.Request(url=next_page, callback=self.parse1)
            
    def parse2(self, response):
        
        company = response.css("section.job-summary li[class='company icon'] div a::text").get()

        description = BeautifulSoup("".join(response.css("div.job-description p").getall()), 'html.parser')
        
        salary_details = response.css("section.job-summary li[class='salary icon'] div::text").get()
        
        job_type = response.css("section.job-summary li[class='job-type icon'] div::text").get()
        
        yield {"URL": response.url, "Company-Recruiter": company, "Description": description.get_text(), "Salary": salary_details, "Job_Type": job_type}

In [ ]:
# %%writefile src/ocean_jobs/data/crawl_totaljobs.py
from scrapy.crawler import CrawlerProcess
# from ocean_jobs.data.spiders.totaljobs import JobSpider

In [ ]:
# %%writefile -a src/ocean_jobs/data/crawl_totaljobs.py

def run_crawler(output_file):
    process = CrawlerProcess({
        'USER_AGENT': 'Mozilla/4.0 (compatible; MSIE 7.0; Windows NT 5.1)',
        'FEED_FORMAT': 'csv',
        'FEED_URI': output_file
    })
    process.crawl(JobSpider)
    process.start() # the script will block here until the crawling is finished


In [ ]:
# %%writefile -a src/ocean_jobs/data/crawl_totaljobs.py

if __name__ == '__main__':
    output_file = 'data/raw/totaljobs.csv'
    run_crawler(output_file)